<a href="https://colab.research.google.com/github/ganjiakhil/Information-Retrieval-Assignments/blob/main/IR_2021_Assignment_1_LSH_with_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Locality Sensitive Hashing

In this assignment you will implement localitiy sensitive hashing step-by-step and apply that on detecting almost duplicate names. You are required to implement the functions as instructed. Do not change the signatures of the functions.

# Deadline: Monday, 7 June 2021 03:00 AM 
That's the night of Sunday, 6 June 2021. Think of it as the Sunday night, with just a few hours more in case you want to be up late. 

# Deadline extended to Thursday, 10 June 3 AM
That's final, no more extension.

## Submission instruction

Submit an ipynb file to the classroom. If you work on Colab, download your file as ipynb and upload that file. Name your file suffixed by your roll number for convenience. 

<b>Important:</b> Do not just share the link of your colab file. Since the assignment needs to be submitted before the deadline, the file you submit should not be editable by you after you submit. A mistake in this regard will not be acceptable under any circumstances, it will be equivalent to not submitting the assignment. 

<b>Late submission policy:</b> For every hour of being late, you will be charged 1% of the marks you get. So, if you submit 2 days late, you lose almost half of the marks. Please do not keep this as a last minute task, you will need time to solve it. 

## Important: do not change the function signatures

You need to implement the functions given in this notebook. You may define more functions at your will if you need to, but make sure you implement the functions given and do not change their signatures. Before submitting, remove unnecessary print / collect statements in order to avoid too much console output. 

## Setup environment

We need to setup pyspark and pydrive. 

In [5]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.4MB 71kB/s 
     |████████████████████████████████| 204kB 21.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=74cdfce61d48a9162c16f4dc5f13b3517a840920c16f65aa1053b7c69ae11695
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

Follow the interactive instructions for accessing the file in Google drive. 

In [6]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

### Load the appropriate file

The files are shared on Google drive, with the following ids given. Choose to keep the appropriate line, use the corresponding filename and comment the others. After executing, you should be able to see the file in the files section on the left panel. 

<b>You may face troubles trying to run your code on the full data, so it is totally okay to run it on one of the smaller files.</b>

In [7]:
# filename = 'names-small.txt'
filename = 'names.txt'
# id='1DrFaE8B8CEHlr17JcN0TAWB8kBayQhjx' # for names-small.txt
id='1oKRbRimRlAkKSya0CFiR04uACLPK7UqK' # for names.txt 

# Test that it's working
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile(filename)

## Parameters

Let us set our parameters. You are free to change these around and experiment. However, do not hardcode them anywhere else in the code. 

In [8]:
# Number of hash functions
n = 20
# size of each shingle
k = 3
# Number of bands 
b = 4

### 1. Converting a string to shingles (of characters) [Marks: 1] 

In our setup, the items we would compare are names (strings). You would have to convert names to sets of k-shingles (of characters), after <i>converting to lowercase</i>. For example, the 3-shingling of the name <tt>'alexander'</tt> would be <tt>['ale', 'lex', 'exa', 'xan', 'and', 'nde', 'der']</tt>.

Implement the following function which takes a string and k as input and outputs the list of unique shingles generated from the string. 

In [3]:
def string_to_shingles(name, k):
  # Implement the code here and return appropriately
  shingles = []
  size = len(name)
  if k == 0 or k > size :
    return []
  for i in range(size):
    shingles.append(name[i:i+k])
    if i == size-k :
      break
  return shingles

Also test your code. 

In [4]:
print(string_to_shingles('Alexander',3))

['Ale', 'lex', 'exa', 'xan', 'and', 'nde', 'der']


### 2. Generating the shingle - item matrix from the whole data [Marks: 3]

For each name, first give it an ID. You can do it by the <tt>zipWithIndex()</tt> operation on Spark RDDs. Try testing it first. 

Then, for each name, get the list of shingles in map and for each unique shingle, get the list of IDs using reduce. 

Tips: when your input RDD is a list of tuples of the following form:

<tt>
    [('alexander', 0),
 ('alexandar', 1),
 ('aleksander', 2),
 ('aleksandar', 3), .... ]
</tt>

for each tuple <tt>t</tt>, you may use <tt>t[0], t[1]</tt> etc to access the first, second (and so on) elements of the tuples. 

In [9]:
from pyspark import SparkContext, SparkConf
sc = SparkContext.getOrCreate()

# Use appropriate file path here
names = sc.textFile(filename)

# Get the number of names
N = names.count()

print(N)

19948


In [126]:
namesWithIndex = names.zipWithIndex()

# Implement the rest of the code so that it produces the RDD namesByShingles in the format required
tuples = namesWithIndex.collect()
total_shingles = []
for t, i in tuples:
   shingles = string_to_shingles(t.lower(), k)
   shingle_tuples = list(zip(shingles, [i]*len(shingles)))
   total_shingles.extend(shingle_tuples)

'''
def pairing(list_of_tuples):
  output = {}
  for t in list_of_tuples:
    try:
      output[t[0]].append(t[1])
    except KeyError:
      output[t[0]] = [t[1]]
  return [(x, y) for x, y in output.items()]

namesByShingles = pairing(total_shingles)

'''

shingles_rdd = sc.parallelize(total_shingles)
namesByShingles = shingles_rdd.groupByKey().map(lambda x : (x[0], list(x[1])))

# Get the number of shingles 
# (for testing with small data, should not do this for large data)
m = namesByShingles.count()
print(m)

5163


In [127]:
# Testing
data = namesByShingles.collect()
print(data)

[('aña', [0, 3295, 5786, 12895, 12896, 16853]), ('ñat', [0, 2453]), ('ter', [0, 575, 969, 1508, 1509, 1995, 1996, 3091, 3267, 3315, 3316, 3317, 3599, 4608, 4667, 5065, 5079, 5253, 5254, 5319, 5320, 5546, 5547, 5834, 5835, 5836, 5856, 5861, 5862, 5943, 6473, 6524, 7201, 7220, 7270, 7786, 8042, 9043, 9872, 9873, 9874, 9875, 10773, 11915, 14470, 14503, 14546, 14547, 14619, 14774, 14775, 16851, 16852, 16853, 16854, 16990, 16991, 16992, 17190, 17194, 17459, 17460, 17461, 17462, 17463, 17464, 17465, 17466, 17467, 17468, 17469, 17470, 17471, 17472, 17473, 17474, 17475, 17476, 17477, 17478, 17479, 17480, 17481, 17482, 17483, 17484, 17485, 17486, 17487, 17488, 17489, 17490, 17491, 17492, 18063, 18467, 18674, 18694, 19173]), ('erv', [0, 5756, 5757, 5836, 6861, 6862, 6863, 7784, 12513, 12514, 12704, 12705, 12706, 14466, 14467, 16205, 16206, 16207, 16208, 16209, 16210, 16211, 16212, 16213]), ('aad', [2, 1909, 5798, 12809, 12942, 15661, 15662, 15663, 15664, 16856]), ('aal', [3, 4, 5, 36, 471, 8847]

At the end of this step, the <tt>namesByShingles</tt> RDD should have the list of (shingle, [list of name ids]).

If you display your output, it should look like:

[(shingle, [list of name ids]), (shingle, [list of name ids]), ... ]

<b>However, do not keep collect or large print statements in your code at the time of submitting.</b>

### 3. Computation of min-hash signature matrix [Marks: 2+2=4]

Instead of using random permutations, you will implement min-hash function using Murmurhash (v3) functions, as discussed in the class. The input to your function should be a <i>title</i> (which corresponds to a row) and the output should be a number (hash value). 

Recall the outline of the min-hash signature matrix algorithm:<br><br>

<tt>
1. For each row $r$ BEGIN<br>
2. &emsp;Compute $h_1(r), h_2(r),…, h_n(r)$<br>
3. &emsp;  For each column $j$ BEGIN <br>
4. &emsp; &emsp;  If the $j$-th column has 1 in row $r$ <br>
5. &emsp;&emsp;&emsp; For each $i = 1, 2, … , n$ BEGIN <br>
6. &emsp;&emsp;&emsp;&emsp; set $\mathrm{SIG}_{i,j} = \min(\mathrm{SIG}_{i,j}, h_i(r))$<br>
7. &emsp;&emsp;&emsp;            END <br>
8. &emsp;&emsp;        END IF<br>
9. &emsp;  END<br>
10. END
</tt>

For each shingle, we have the list of name IDs which contain the shingle. So, for each shingle (each row), we can perform the actions in lines 5-7 only for the name IDs that are present in the list. For any other $j$, the $j$-th column does not have a 1 in row $r$. 

Do not create a SIG matrix and try to update it. Instead, note that for every 1-entry in the shingle-id matrix (that is, every id in the list corresponding to a shingle), the corresponding (i, id)-th entry of the signature matrix may get potentially updated. Simply output the tuple ((i, id), h_i(text)) in another map. The final value of the (i,id)-th entry of the signature matrix is the minimum of all such h_i(text) values obtained in this process. That minimum can be computed by another reduce process. 

### (a) The function for map [Marks: 2 out of 4]
To make it easier, you may implement an <tt>update_signature</tt> function which takes the shingle (text), the list of ids associated with the shingle and the total number of shingles (for computing the hash values). It should simply return the tules <tt>((i, id), h_i(text))</tt> for all i = 1, ..., n. 

In [55]:
pip install mmh3

     |████████████████████████████████| 51kB 4.5MB/s 


In [128]:
import mmh3

def update_signature(text, ids, numOfShingles):
    
  h = [mmh3.hash(text,i) % (numOfShingles) for i in range(n)]
  min_sig = []

  # Implement the function
  for i in range(n):
    for j in ids:
      min_sig.append(((i, j), h[i]))

  return min_sig

In [129]:
# testing
sh_ids = namesByShingles.collect()
print(update_signature(sh_ids[0][0], sh_ids[0][1], m))

[((0, 0), 4810), ((0, 3295), 4810), ((0, 5786), 4810), ((0, 12895), 4810), ((0, 12896), 4810), ((0, 16853), 4810), ((1, 0), 4526), ((1, 3295), 4526), ((1, 5786), 4526), ((1, 12895), 4526), ((1, 12896), 4526), ((1, 16853), 4526), ((2, 0), 2323), ((2, 3295), 2323), ((2, 5786), 2323), ((2, 12895), 2323), ((2, 12896), 2323), ((2, 16853), 2323), ((3, 0), 942), ((3, 3295), 942), ((3, 5786), 942), ((3, 12895), 942), ((3, 12896), 942), ((3, 16853), 942), ((4, 0), 2543), ((4, 3295), 2543), ((4, 5786), 2543), ((4, 12895), 2543), ((4, 12896), 2543), ((4, 16853), 2543), ((5, 0), 4937), ((5, 3295), 4937), ((5, 5786), 4937), ((5, 12895), 4937), ((5, 12896), 4937), ((5, 16853), 4937), ((6, 0), 3022), ((6, 3295), 3022), ((6, 5786), 3022), ((6, 12895), 3022), ((6, 12896), 3022), ((6, 16853), 3022), ((7, 0), 1528), ((7, 3295), 1528), ((7, 5786), 1528), ((7, 12895), 1528), ((7, 12896), 1528), ((7, 16853), 1528), ((8, 0), 1509), ((8, 3295), 1509), ((8, 5786), 1509), ((8, 12895), 1509), ((8, 12896), 1509),

### (b) The map and reduce [Marks: 2 out of 4]

Now that you have all <tt>((i, id), h_i(shingle))</tt> tuples output by the map, use reduce to compute the minimum of <tt>h_i(shingle)</tt> for every <tt>(i,id)</tt> key. This would produce the signature matrix in a sparse matrix representation. 

You may actually implement the map function above and call map and reduce together later as well. 

In [130]:
numOfShingles = m # Calculate, you need to pass this to the update_signature function

shingle_ids = namesByShingles.collect()
signature_list = []

for t in shingle_ids:
  signature_list.extend(update_signature(t[0], t[1], m))

signature_rdd = sc.parallelize(signature_list)
signature = signature_rdd.groupByKey().map(lambda x : ((x[0][0], x[0][1]), min(list(x[1]))))

# You should use map with the update_signature function and a reduce 

signature.take(5)

[((0, 0), 469),
 ((0, 5786), 71),
 ((0, 12896), 834),
 ((1, 3295), 555),
 ((1, 12895), 1133)]

In [131]:
# Testing 
c = signature.count()
print(c)

397400


The RDD signature should be of the following form:

<tt>
    [((0, 1), 5), ((0, 3), 56), ... 
    ]
</tt>

where each tuple <tt>((i,j),v)</tt> represents the $(i,j)$-th entry of the signature matrix with value $v$.

### 4. Implement the banding [Marks: 2]

Now configure your number of bands $b$ (a divisor of number of hash functions $n$) and implement the candidate pair computation. Any pair of names which agree completely in their signature on at least one band should be output as candidate pairs. 

In [132]:
number_of_bands = 5 # Choice of bands is user defined and a divisor of number of hash functions



candidate_pairs = []

The candidate pairs should be of the form

<tt>
    [(name1, name2), (name3, name2), ... ]
</tt>

### 5. The Banding (ungraded - optional): 

Optionally, you may also want to compute the pairwise actual Jaccard similarities for each candidate pair and test the number of false positives and negatives for a similarity threshold $s$. 